# Anime TV Show Recommender

In [ ]:
import pandas as pd

In [7]:
userRatings = pd.read_csv('rating_complete.csv', encoding='latin-1')
userRatings.head()

,user_id,anime_id,rating
0,0,430,9
1,0,1004,5
2,0,3010,7
3,0,570,7
4,0,2762,9


In [8]:
animeList = pd.read_csv('anime.csv', encoding='latin-1')
animeList.head()

,MAL_ID,Name,Score,Genres,English name,Japanese name,Type,Episodes,Aired,Premiered,...,Score-10,Score-9,Score-8,Score-7,Score-6,Score-5,Score-4,Score-3,Score-2,Score-1
0,1,Cowboy Bebop,8.78,"Action, Adventure, Comedy, Drama, Sci-Fi, Space",Cowboy Bebop,ã«ã¦ãã¼ã¤ãããã,TV,26,"Apr 3, 1998 to Apr 24, 1999",Spring 1998,...,229170.0,182126.0,131625.0,62330.0,20688.0,8904.0,3184.0,1357.0,741.0,1580.0
1,5,Cowboy Bebop: Tengoku no Tobira,8.39,"Action, Drama, Mystery, Sci-Fi, Space",Cowboy Bebop:The Movie,ã«ã¦ãã¼ã¤ãããã å¤©å½ã®æ,Movie,1,"Sep 1, 2001",Unknown,...,30043.0,49201.0,49505.0,22632.0,5805.0,1877.0,577.0,221.0,109.0,379.0
2,6,Trigun,8.24,"Action, Sci-Fi, Adventure, Comedy, Drama, Shounen",Trigun,ãã©ã¤ã¬ã³,TV,26,"Apr 1, 1998 to Sep 30, 1998",Spring 1998,...,50229.0,75651.0,86142.0,49432.0,15376.0,5838.0,1965.0,664.0,316.0,533.0
3,7,Witch Hunter Robin,7.27,"Action, Mystery, Police, Supernatural, Drama, ...",Witch Hunter Robin,Witch Hunter ROBIN (ã¦ã¤ãããã³ã¿ã¼ã...,TV,26,"Jul 2, 2002 to Dec 24, 2002",Summer 2002,...,2182.0,4806.0,10128.0,11618.0,5709.0,2920.0,1083.0,353.0,164.0,131.0
4,8,Bouken Ou Beet,6.98,"Adventure, Fantasy, Shounen, Supernatural",Beet the Vandel Buster,åéºçãã£ã,TV,52,"Sep 30, 2004 to Sep 29, 2005",Fall 2004,...,312.0,529.0,1242.0,1713.0,1068.0,634.0,265.0,83.0,50.0,27.0


In [13]:
animeReviewSum = animeList
animeReviewSum['Total # of Reviews'] = animeList.iloc[:, 11:20].sum(axis=1)
animeReviewClean =  animeReviewSum.iloc[:, [0,4,-1]]

#Only shows with over 100000 reviews

animeListClean = animeReviewClean[animeReviewClean['Total # of Reviews'] > 100000] 
animeListClean.head()

C:\Users\vinai\AppData\Local\Temp\ipykernel_7024\877895963.py:2: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  animeReviewSum['Total # of Reviews'] = animeList.iloc[:, 11:20].sum(axis=1)


,MAL_ID,English name,Total # of Reviews
0,1,Cowboy Bebop,1313970
1,5,Cowboy Bebop:The Movie,274837
2,6,Trigun,572058
5,15,Unknown,151328
6,16,Honey and Clover,219287


In [67]:
validID = animeListClean['MAL_ID']

# Filter the userRating DataFrame based on valid MAL_IDs
userRatingsClean = userRatings[userRatings['anime_id'].isin(validID)]

userRatingsClean.head()

,user_id,anime_id,rating
0,0,430,9
3,0,570,7
5,0,431,8
6,0,578,10
7,0,433,6


In [68]:
#Clean users with only user with over 15 reviews 
userReviewCounts = userRatingsClean['user_id'].value_counts()
highReviewCounts = userReviewCounts[userReviewCounts >=15]
userCleaned =  userRatingsClean[userRatingsClean['user_id'].isin(highReviewCounts)]

userCleaned.head(100)
#userCleaned.shape[0]

,user_id,anime_id,rating
1427,15,482,7
1428,15,527,10
1430,15,72,7
1432,15,225,5
1433,15,223,10
...,...,...,...
1543,16,237,7
1544,16,10790,7
1545,16,8841,6
1546,16,38040,7


In [69]:
print("Review Counts for Each User:")
print(userCleaned)

Review Counts for Each User:
        user_id  anime_id  rating
1427         15       482       7
1428         15       527      10
1430         15        72       7
1432         15       225       5
1433         15       223      10
...         ...       ...     ...
219151     1373       177       5
219152     1373      4224       6
219153     1373       949       6
219155     1373     10162       8
219156     1373       523       9

[122254 rows x 3 columns]


# User-Item Factorization

In [88]:
userItemMatrix = userCleaned.pivot_table(index='user_id', columns='anime_id', values='rating')
userItemMatrix = userItemMatrix.fillna(0)
userItemMatrix.head()

anime_id,1,5,6,15,16,18,19,20,22,24,...,41168,41226,41345,41353,41380,41389,41433,41468,41619,41930
user_id,,,,,,,,,,,,,,,,,,,,,
15,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
16,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.0,0.0,7.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
17,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
18,0.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
19,8.0,6.0,8.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# SVD and NMF Model 

In [89]:
from surprise import Dataset, Reader
from surprise import SVD
from surprise import NMF
from surprise.model_selection import train_test_split
from surprise.model_selection import cross_validate
from surprise import accuracy

In [90]:
userItemMatrix.reset_index(inplace=True)
surpriseDF = pd.melt(userItemMatrix, id_vars=['user_id'], var_name='anime_id', value_name='rating', ignore_index=True)

reader = Reader(rating_scale=(0, 10))
data = Dataset.load_from_df(surpriseDF[['user_id', 'anime_id', 'rating']], reader)

svdModel = SVD()
svdResults = cross_validate(svdModel, data, measures=['RMSE'], cv=5, verbose=True)

print("SVD Mean RMSE:", svdResults['test_rmse'].mean())

Evaluating RMSE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    1.8813  1.8732  1.8762  1.8841  1.8699  1.8770  0.0052  
Fit time          12.59   12.94   12.92   12.91   12.99   12.87   0.14    
Test time         2.64    3.70    2.61    2.56    3.64    3.03    0.52    
SVD Mean RMSE: 1.8769564887435812


In [91]:
nmfModel = NMF()
nmfResults = cross_validate(nmfModel, data, measures=['RMSE'], cv=5, verbose=True)
print("NMF Mean RMSE:", nmfResults['test_rmse'].mean())

Evaluating RMSE of algorithm NMF on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    1.9221  1.8996  1.9054  1.9105  1.9204  1.9116  0.0086  
Fit time          22.06   22.93   22.48   23.10   22.52   22.62   0.37    
Test time         2.25    3.52    2.19    3.53    2.19    2.73    0.64    
NMF Mean RMSE: 1.9116296093955583
